# Домашнее задание 1

**Выполнил:** Оникова Дарья Тимуровна

**На сколько выполнил:** <Укажите, на сколько баллов по вашему мнению вы сделали>

**Правила игры:**

* возле каждой задачи указано число баллов (в сококупности можно получить 10 баллов)

* дополнительные задачи выделены звездочкой (также указано число баллов). Данные баллы можно будет использовать для улучшения результата за любое домашнее задание

* все подсчеты необходимо делать с помощью pandas-numpy (использовать как можно меньше циклов, за использование циклов, где это не требуются, будем штрафовать)

* для визуализации можно использовать matplotlib-seaborn-plotly (учтите, что все графики должны выглядеть опрятно, иметь название-подписи, за нечитабельные графики будем штрафовать)

* возможно, что в данных есть пустоты и невалидные значения. Перед началом любой работы необходимо исследовать, и если такие есть, убрать их

* соблюдайте чистоту кода: переменные должны быть осмысленными, не повторяйтесь, если в этом нет необходимости (иначе используйте функции). Чем чище код - тем проще нам поставить вам хорошую оценку :)

* общая рекомендация: на вопросы со звездочкой отвечать в самом конце, возможно, будет проще)

## Данные 

В этом домашнем задании вам придется оказаться на месте аналитика в бразильском маркетплейсе [Olist](https://olist.com/pt-br/). Вам необходимо исследовать данные и на их основании сделать выводы, которые помогут бизнесу расцветать!

Данные находятся тут: (будет ссылка)

Ван дано 9 датасетов, которые содержат в себе все данные по 100 000 заказам со всей Бразилии. Чтобы облегчить вам жизнь, вот связи по этим датасетам (файл product_category_name_translation является переводом названий категорий с португальского на английский)

![](https://i.imgur.com/HRhd2Y0.png)

Ну что же, начнем, пожалуй!

In [1]:
# imports 
import numpy as np
import pandas as pd
import numpy.linalg as lin
import scipy.stats as st
import scipy.sparse as sp
import matplotlib.pyplot as plt
import dateutil as dt
import datetime as datetime

In [144]:
# plot imports 

import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots


# Подготовка данных

In [2]:
!unzip archive.zip

Archive:  archive.zip
  inflating: olist_customers_dataset.csv  
  inflating: olist_geolocation_dataset.csv  
  inflating: olist_order_items_dataset.csv  
  inflating: olist_order_payments_dataset.csv  
  inflating: olist_order_reviews_dataset.csv  
  inflating: olist_orders_dataset.csv  
  inflating: olist_products_dataset.csv  
  inflating: olist_sellers_dataset.csv  
  inflating: product_category_name_translation.csv  


In [83]:
customers = pd.read_csv('olist_customers_dataset.csv')
geolocation = pd.read_csv('olist_geolocation_dataset.csv')
order_items = pd.read_csv('olist_order_items_dataset.csv')
order_payments = pd.read_csv('olist_order_payments_dataset.csv')
order_reviews = pd.read_csv('olist_order_reviews_dataset.csv')
orders = pd.read_csv('olist_orders_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')
sellers = pd.read_csv('olist_sellers_dataset.csv')
product_names = pd.read_csv('product_category_name_translation.csv')

In [22]:
def clear_data(x):
    x = x.dropna()
    return x

In [84]:
customers = clear_data(customers)
geolocation = clear_data(geolocation)
order_items = clear_data(order_items)
order_reviews = clear_data(order_reviews)
order_payments = clear_data(order_payments)
orders = clear_data(orders)
products = clear_data(products)
sellers = clear_data(sellers)
product_names = clear_data(product_names)

In [85]:
products.product_category_name.replace(list(product_names["product_category_name"]),
                                       list(product_names["product_category_name_english"]), inplace=True)

In [86]:
products.set_index('product_id', inplace=True)

## Задание 1 (0.4 балла)

Определите:

* Число товаров
* Среднюю стоимость товара

в разрезе категорий (все категории должны быть на английском языке)

In [101]:
prices = order_items[["product_id", "price"]].set_index("product_id").groupby('product_id').mean()
prices = pd.concat([prices, products["product_category_name"]], axis=1)
prices.head()

,price,product_category_name
product_id,,
00066f42aeeb9f3007548bb9d3f33c38,101.65,perfumery
00088930e925c41fd95ebfe695fd2655,129.90,auto
0009406fd7479715e4bef61dd91f2462,229.00,bed_bath_table
000b8f95fcb9e0096488278317764d19,58.90,housewares
000d9be29b5207b54e86aa1b1ac54872,199.00,watches_gifts


In [104]:
amount = order_items.set_index("product_id").groupby('product_id').size()
amount = pd.concat([amount, products["product_category_name"]], axis=1)
amount = amount.rename(columns={0:"amount"})
amount.head()

,amount,product_category_name
product_id,,
00066f42aeeb9f3007548bb9d3f33c38,1,perfumery
00088930e925c41fd95ebfe695fd2655,1,auto
0009406fd7479715e4bef61dd91f2462,1,bed_bath_table
000b8f95fcb9e0096488278317764d19,2,housewares
000d9be29b5207b54e86aa1b1ac54872,1,watches_gifts


число товаров:

In [106]:
amount.groupby("product_category_name").count()

,amount
product_category_name,
agro_industry_and_commerce,74
air_conditioning,124
art,55
arts_and_craftmanship,19
audio,58
...,...
stationery,849
tablets_printing_image,9
telephony,1134


цена в разрезе категорий


In [108]:
prices.groupby("product_category_name").mean()

,price
product_category_name,
agro_industry_and_commerce,438.692969
air_conditioning,205.868773
art,195.019858
arts_and_craftmanship,65.839474
audio,116.488832
...,...
stationery,103.571095
tablets_printing_image,163.684210
telephony,83.826234


## Задание 2 (1,5 балла)

Определите для каждого продавца основную категорию их продаж.

Отобразите категории по числу продавцов, для которых данная категория основная, а также создайте график, по которому можно выбрать отдельно штат и посмотреть такое же распределение



In [113]:
sellers_cat_states = order_items[["seller_id", "product_id"]]
sellers_states = sellers[["seller_id", "seller_state"]]
sellers_cat_states.head()

KeyError: "['seller_state'] not in index"

## Задание 3 (1,5 балл)

Покажите процент

* суммы покупок в деньгах

* суммы покупок в штуках

по категории для доставленных заказов, а также разбивку по штатам

## Задание 4 (0.6 балла)

Визуализируйте зависимость между средним числом покупок и:

* числом фотографий товара

* кол-во символов в описании товара (аггрегируйте с шагом 20)

* кол-во символов в названии (аггрегируйте с шагом 5)

In [141]:
sales_stats = pd.concat([products[["product_photos_qty", "product_description_lenght", "product_name_lenght"]], order_items[["product_id"]].groupby("product_id").size()], axis=1)
sales_stats = sales_stats.rename(columns={0: "amount"})
sales_stats.head()

,product_photos_qty,product_description_lenght,product_name_lenght,amount
product_id,,,,
1e9e8ef04dbcff4541ed26657ea517e5,1.0,287.0,40.0,1
3aa071139cb16b67ca9e5dea641aaa2f,1.0,276.0,44.0,1
96bd76ec8810374ed1b65e291975717f,1.0,250.0,46.0,1
cef67bcfe19066a932b7673e239eb23d,1.0,261.0,27.0,1
9dc1a7de274444849c219cff195d0b71,4.0,402.0,37.0,1


In [165]:
sales_stats_photos = sales_stats.groupby("product_photos_qty").mean()[["amount"]]
sales_stats_descr = sales_stats.groupby("product_description_lenght").mean()[["amount"]]
sales_stats_name = sales_stats.groupby("product_name_lenght").mean()[["amount"]]

In [160]:
pd.options.plotting.backend = "plotly"

In [162]:
fig1 = sales_stats_photos.plot()
fig1.update_layout(title=" зависимость между средним числом покупок и числом фотографий товара",
                  xaxis_title="число фотографий",
                  yaxis_title="среднее число покупок")
fig1.update_layout(showlegend=False)
fig1.show()

In [163]:
fig2 = sales_stats_descr.plot()
fig2.update_layout(title=" зависимость между средним числом покупок и длиной описания",
                  xaxis_title="длина описания",
                  yaxis_title="среднее число покупок")
fig2.update_layout(showlegend=False)
fig2.show()

In [166]:
fig3 = sales_stats_name.plot()
fig3.update_layout(title="зависимость между средним числом покупок и длиной названия",
                  xaxis_title="длина названия",
                  yaxis_title="среднее число покупок")
fig3.update_layout(showlegend=False)
fig3.show()

## Задание 5* (1 балл)

Имея полученные данные по товарам, чекам и категориям, а также сделав дополнительные расчеты (если необходимо), предположите:

1. Продавцов каких категорий необходимо привлекать в маркетплейс?

2. Какие категории в каких штатах необходимо развивать?

3. Развитие каких категорий не принесут большого эффекта и почему?

4. Какие рекомендации можно дать для продавцов, чтобы их товар лучше продавался?

## Задание 6 (0.5 балла)

Определите средний чек покупки (добавьте разбивку на стоимость самого заказ и стоимость доставки) и среднее число товаров в заказе

## Задание 7 (0.2 балла)

Определите среднее число покупок на пользователя (обратите внимание на идентификаторы)

## Задание 8 (0.2 балла)

Отобразите среднюю сумму оплаты по типу оплаты

## Задание 9 (0.4 балла)

Обратите внимание на число платежей (payment_installments). Выясните, какая сумма оплаты указана: за целую покупку или только за ее часть?

Также покажите средний чек в зависимости от числа оплаты. Есть ли какая-то связь между ними?

## Задание 10 (0.4 балла)

Визуализируйте зависимость между ценой заказа и ценой доставки по штатам покупки. Есть ли какая-нибудь зависимость и как вы можете ее обосновать?

## Задание 11 (0.4 балла)

Выведите график по числу покупок (в разбивке по статусам) и временем покупки (аггрегируйте покупки до дня)

## Задание 12* (1 балл)

Используя полученные данные, а также сделав дополнительные вычисления (если необходимо), предположите:

1. Есть ли сезонность в покупках на маркетплейсе?

2. Есть ли какие-то необъяснимые падение/рост продаж? С чем это может быть связано?

## Задание 13 (0.3 балла)

Посчитайте CSAT (customer satisfaction - средняя оценка ревью) и отобразите средний CSAT по дням



## Задание 14 (0.3 балла)

Найдите топ-5 лучших и топ-5 худших продавцов по средней оценке и посчитайте их число продаж

## Задание 15 (0.3 балла)

Посмотрите, как быстро отвечают пользователи (сделайте аггреграцию по дням) и влияет ли это на оценку?

## Задание 16 (0.5 балла)

Отобразите распределение оценок для заказов, которые были доставлены вовремя и которые пришли с опозданием. Насколько влияет факт опоздания на оценку?

## Задание 17* (1,5 балла)

Есть ли среди пользователей хейтеры? Определите их и вычислите средний CSAT без их участия

## Задание 18 (0.4 балла)

Изобразите зависимость между ценой заказа и числом дней между покупкой и доставкой с разбивкой по признаку "есть заказ из другого штата". Есть ли зависимость?

## Задание 19 (0.6 баллов)

Отобразите на карте все точки продавцов (если они находятся в одном месте, то необходимо отметить кол-во)

Где больше всего продавцов находится?

## Задание 20 (1,5 балл)

Определите топ-5 продавцов, которые чаще всего отсылают свою посылку в другие регионы и визуализируйте их отсылки

## Задание 21* (2 балла)

Сделайте анимацию по датам с доставкой между продавцами и пользователями

## Задание 22* (творческое, отдельно поставим баллы)

Проведите дополнительно исследование между параметрами, которые вам интересны и могут быть полезными (обоснуйте, зачем это смотреть). Сделайте выводы

## Попугай для ДЗ

![](https://upload.wikimedia.org/wikipedia/commons/thumb/c/ca/Rose-ringed_parakeet_%28Psittacula_krameri_manillensis%29.jpg/1024px-Rose-ringed_parakeet_%28Psittacula_krameri_manillensis%29.jpg)

А это ожереловый попугай (или индийский кольчатый попугай). Этот вид явно выделяет наличие такого колечка на шее. Самый распространенный попугай в мире

Они очень хорошо и быстро летают, но ходят максимально неуклюже)

В отличии от многих попугаев, для которых человеческая деятельность является вредом и уменьшает их естественный ареал обитания, то как раз для этих птиц человеческая деятельность - это плюс, потому что они находят корм внутри человеческих обществ (не боятся людей, лучше всего приспосабливаются, едят пшеницу). Поговаривают, что в качестве домашних попугаев их держали еще в древней Греции и Риме, а в  средневековой Индии ожереловые попугаи часто были питомцами знатных особ и монархов, причём владение этим попугаем считалось элементом роскоши и престижа